In [2]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt


In [3]:
camera = cv2.VideoCapture(1)
count = 0
while True:
    ret, frame = camera.read()
    if ret:
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('s'):
            cv2.imwrite('frame%d.jpg' % count, frame)
            count += 1
            pass
        if cv2.waitKey(1) & 0xFF == ord('p'):
            pass
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cv2.destroyAllWindows()
camera.release()

In [6]:
imgL = cv2.imread('tsukuba_l.png',0)
imgR = cv2.imread('tsukuba_r.png',0)

stereo = cv2.StereoBM_create(numDisparities=16, blockSize=15)   
disparity = stereo.compute(imgL,imgR)
#create a normal map from the disparity image
normal_map = cv2.normalize(disparity, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
#show the normal map
cv2.imshow('normal map', normal_map)
cv2.save('normal_map.png', normal_map)


In [48]:
#create point cloud
h, w = imgL.shape[:2]
points = np.zeros((h, w, 3), np.float32)
for y in range(h):
    for x in range(w):
        points[y, x] = (x, y, disparity[y, x])

#convert to 3d point cloud
points = points.reshape(-1, 3)
#save point cloud
header = '''# .PCD v.7 - Point Cloud Data file format
FIELDS x y z
SIZE 4 4 4
TYPE F F F
COUNT 1 1 1
WIDTH %d
HEIGHT 1
VIEWPOINT 0 0 0 1 0 0 0
POINTS %d
'''
with open('point_cloud.pcd', 'w') as f:
    f.write(header % (len(points), len(points)))
    np.savetxt(f, points, fmt='%f %f %f')
    print('point cloud saved')

point cloud saved
